In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from dotenv import load_dotenv, find_dotenv
import nest_asyncio
import warnings

nest_asyncio.apply()
warnings.filterwarnings("ignore")
_ = load_dotenv(find_dotenv())

In [21]:
from llama_index.core.agent import ReActAgent
from llama_index.core.llms import ChatMessage
from llama_index.core.llms.llm import LLM
from llama_index.core.memory import ChatMemoryBuffer
from llama_index.core.program import LLMTextCompletionProgram
from llama_index.core.workflow import (
    Event,
    StartEvent,
    StopEvent,
    Context,
    Workflow,
    step
)
from llama_index.core.workflow.context_serializers import (
    JsonSerializer, 
    JsonPickleSerializer
)
from llama_index.core.workflow.events import (
    InputRequiredEvent,
    HumanResponseEvent,
)
from llama_index.llms.gemini import Gemini

In [15]:
from pydantic import BaseModel, Field
from typing import List

class Exercise(BaseModel):
    """Generates a workout plan given a focus area"""
    exercise: str
    number_of_reps: int
    number_of_sets: int

class WorkoutPlan(BaseModel):
    exercises: List[Exercise]

class InputNeeded(BaseModel):
    input_needed: bool = Field(default=False, description="Whether further clarification is needed")
    reason: str = Field(..., description="Reason for deciding if further input is needed.")
    follow_up_question: str | None = Field(default=None, description="The follow-up question to seek clarification")

In [16]:
llm = Gemini(model="models/gemini-1.5-flash")

prompt_template_str = """\
You are the customer success agent and your task is to ensure the customer's success \ 
in achieving their fitness and health goals. \
This is the customer's query, determine if more clarification is needed. \
Return either true or false with the reason. \
Here is the user's query: {query}.\
"""

In [17]:
from llama_index.core.program import LLMTextCompletionProgram

program = LLMTextCompletionProgram.from_defaults(
    llm = llm,
    output_cls=InputNeeded, 
    prompt_template_str=prompt_template_str,
    verbose=True,
)

output = program(
    query="I want to get fit."
)

In [18]:
output

InputNeeded(input_needed=True, reason="The customer's goal is very broad. To help them, I need to understand what 'fit' means to them.  What are their specific goals?  Do they want to lose weight, gain muscle, improve endurance, or something else?", follow_up_question="Can you tell me more about what you mean by 'getting fit'? What are your specific goals?")

In [14]:
output.input_needed

True

In [23]:
class GoalSettingEvent(Event):
    query: str

class WorkoutPlannerEvent(Event):
    query: str

class NutritionGuideEvent(Event):
    query: str
    workout_plan: WorkoutPlan
    
class FollowUpEvent(Event):
    query: str

In [4]:
class FitnessWorkflow(Workflow):
    def __init__(
        self, 
        llm: LLM,
        *args,
        **kwargs
    ):
        self.llm = llm
        self.prompt_template_str = """\
        You are the customer success agent and your task is to ensure the customer's success \ 
        in achieving their fitness and health goals. \
        This is the customer's query, determine if more clarification is needed. \
        Return either true or false with the reason. \
        Here is the user's query: {query}.\
        """
        self.program = LLMTextCompletionProgram.from_defaults(
            llm = llm,
            output_cls=InputNeeded, 
            prompt_template_str=prompt_template_str,
            verbose=True,
        )
        self.memory = ChatMemoryBuffer.from_defaults(llm=llm)
        super().__init__(*args, **kwargs)
    
    @step
    async def goal_setting(
        self, 
        ev: StartEvent | FollowUpEvent
    ) -> InputRequiredEvent | WorkoutPlannerEvent:
        self.memory.put(ev.query)
        output = self.program(query=ev.query)
        if output.input_needed is True:   
            self.memory.put(
                ChatMessage(
                    role="assistant",
                    content=output.follow_up_question
                )
            )
            return InputRequiredEvent(prefix=output.follow_up_question)
        self.memory.put(
            ChatMessage(
                role="assistant",
                content=f"This user query is sufficient. Reason: {output.reason}"
            )
        )
        return WorkoutPlannerEvent(query=ev.query)

    @step
    async def judge_clarification(
        self,
        ev: HumanResponseEvent
    ) -> GoalSettingEvent:
        return FollowUpEvent(query=ev.response)
        

In [5]:
final_result

'25'